In [2]:
import numpy as np 
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

In [63]:
def ec_schordinger (x,y,E):
    #y es el vector = [f,f´]
    f,df= y
    ddf = (x**2-2*E)*f
    return [df,ddf]

In [ ]:
def evento_convergencia(x, y, E, threshold=12):
    # y[0] es f(x)
    hipotenusa = np.sqrt(y[0]**2 + y[1]**2)
    return threshold - hipotenusa

In [100]:
def solve_for_energy(E, x_max=6.0):
    y0 = [0.3, 0.0]  # f(0)=1, f'(0)=0 (pares)
    x_span = (0, x_max)
    
    ev_func = lambda x, y: evento_convergencia(x, y, E)
    ev_func.terminal = True
    ev_func.direction = -1

    sol = solve_ivp(
        lambda x, y: ec_schordinger(x, y, E),
        x_span,
        y0,
        rtol=1e-7, 
        atol=1e-9,
        events=lambda x, y: evento_convergencia(x, y, E)   
    )
    
    f_at_end = sol.y[0][-1] 
    event_triggered = (len(sol.t_events[0]) > 0)
    return sol, event_triggered


In [96]:
def solve_for_energy_anti(E, x_max=6.0):
    y0 = [0.0, 1.0]  # f(0)=1, f'(0)=0 (pares)
    x_span = (0, x_max)
    
    ev_func = lambda x, y: evento_convergencia(x, y, E)
    ev_func.terminal = True
    ev_func.direction = -1

    sol = solve_ivp(
        lambda x, y: ec_schordinger(x, y, E),
        x_span,
        y0,
        rtol=1e-7, 
        atol=1e-9,
        events=lambda x, y: evento_convergencia(x, y, E)   
    )
    
    f_at_end = sol.y[0][-1] 
    event_triggered = (len(sol.t_events[0]) > 0)
    return sol, event_triggered

In [97]:
energies_A = np.arange(0.0, 9.52, 0.1)
candidates_A = []  # Aquí se guardarán las energías donde la solución converge

for E in energies_A:
    sol, event_triggered = solve_for_energy_anti(E)
    if not event_triggered:
        candidates_A.append(E)

candidates_A = np.array(candidates_A)
print(candidates_A)

[1.5 3.5 5.5 7.5 9.5]


In [101]:
energies = np.arange(0.0, 10.0, 0.1)
candidates = []  # Aquí se guardarán las energías donde la solución converge

for E in energies:
    sol, event_triggered = solve_for_energy(E)
    if not event_triggered:
        candidates.append(E)

candidates = np.array(candidates)
print(candidates)

[0.5 2.5 4.5 6.5 8.5]


In [ ]:
candidatos= np.concatenate((candidates,candidates_A))

array([0.5, 2.5, 4.5, 6.5, 8.5, 1.5, 3.5, 5.5, 7.5, 9.4, 9.5])

In [120]:
plt.figure(figsize=(6,7))
x=np.linspace(-6,6,200)
plt.plot(x,1/2*x**2,linestyle="--",color="lightgray")

for e in candidates:
    sol, _ = solve_for_energy(e)
    t_negativo = -sol.t[::-1]
    y_negativo= sol.y[0][::-1] 
    t=np.concatenate((t_negativo, sol.t))
    E= np.concatenate((y_negativo, sol.y[0]))
    E= E+e
    plt.axhline(y=e, color='lightgray')
    plt.plot(t, E)

for r in candidates_A:
    sol_a,_= solve_for_energy_anti(r)
    t_n = -sol_a.t[::-1]
    y_n= -sol_a.y[0][::-1]
    t_A= np.concatenate((t_n, sol_a.t))
    E_An= np.concatenate((y_n,sol_a.y[0]))
    E_An= E_An+r
    plt.axhline(y=r, color='lightgray')
    plt.plot(t_A,E_An)


plt.ylim(0,10)
plt.xlim(-6,6)
plt.ylabel("Energía")
plt.savefig("4.pdf", format="pdf", dpi=300, bbox_inches="tight")
plt.clf()

<Figure size 600x700 with 0 Axes>